# Odd Network

You analyze the performance of a computer network. The network comprises nodes connected by peer-to-peer links. There are N links and N + 1 nodes. All pairs of nodes are (directly or indirectly) connected by links, and links don't form cycles. In other words, the network has a tree topology.

Your analysis shows that communication between two nodes performs much better if the number of links on the (shortest) route between the nodes is odd. Of course, the communication is fastest when the two nodes are connected by a direct link. But, amazingly, if the nodes communicate via 3, 5, 7, etc. links, communication is much faster than if the number of links to pass is even.

Now you wonder how this influences the overall network performance. There are N * (N + 1) / 2 different pairs of nodes. You need to compute how many of them are pairs of nodes connected via an odd number of links.

Nodes are numbered from 0 to N. Links are described by two arrays of integers, A and B, each containing N integers. For each 0 ≤ I < N, there is a link between nodes A[I] and B[I].

Write a function:

```
def solution(A, B)
```

that, given two arrays, A and B, consisting of N integers and describing the links, computes the number of pairs of nodes X and Y, such that 0 ≤ X < Y ≤ N, and X and Y are connected via an odd number of links.

EXAMPLE 1

For example, given N = 6 and the following arrays:

```
  A[0] = 0    B[0] = 3
  A[1] = 3    B[1] = 1
  A[2] = 4    B[2] = 3
  A[3] = 2    B[3] = 3
  A[4] = 6    B[4] = 3
  A[5] = 3    B[5] = 5
  
  0  1  2 
   \ | /
     3
   / | \
  4  5  6
```

the function should return 6, since:

  - there are six pairs of nodes connected by direct links, and
  - all other pairs of nodes are connected via two links.


EXAMPLE 2

Given N = 5 and the following arrays:

```
  A[0] = 0    B[0] = 1
  A[1] = 4    B[1] = 3
  A[2] = 2    B[2] = 1
  A[3] = 2    B[3] = 3
  A[4] = 4    B[4] = 5
  
  0 - 1 - 2 - 3 - 4 - 5
```

the function should return 9, since:

  - there are five pairs of nodes connected by direct links,
  - there are three pairs of nodes connected via three links, and
  - there is one pair of nodes connected via five links.

EXAMPLE 3

Given N = 7 and the following arrays:

```
  A[0] = 0    B[0] = 3
  A[1] = 4    B[1] = 5
  A[2] = 4    B[2] = 1
  A[3] = 2    B[3] = 3
  A[4] = 7    B[4] = 4
  A[5] = 6    B[5] = 3
  A[6] = 3    B[6] = 4
  
      0   1
      |   |
  2 - 3 - 4 - 5
      |   |
      6   7
```

the function should return 16, since:

  - there are seven pairs of nodes connected by direct links, and
  - there are nine pairs of nodes connected via three links.

Write an efficient algorithm for the following assumptions:

  - N is an integer within the range [0..90,000];
  - each element of arrays A and B is an integer within the range [0..N];
  - the network has a tree topology;
  - any pair of nodes is connected via no more than 1000 links.

Ref:
- Code challenging from Opus, 2022-10-18.
- https://app.codility.com/cert/view/cert92A85Y-4VM22MZ3A3EC3YE9/details/


In [1]:
from typing import List
from collections import defaultdict
from itertools import combinations


class Solution:
    def solution_v1(self, A: List[int], B: List[int]) -> int:
        """Not efficient, O(N^3)."""
    
        # Build the network: {node: [node]}
        network = defaultdict(set)
        for a, b in zip(A, B):
            network[a].add(b)
            network[b].add(a)
            
        # Use the breath-first method to explore the network
        # Use the queue to list the nodes that need to be processed next.
        # Each element is (start_node, current_node).
        queue = [(k,k) for k in network.keys()] 

        # Use a dictionary to track all of the links that have been processed.
        # (n1, n2) and (n2, n1) are considered different.
        seen = dict()
        num_links = 0
        while queue:
            next_queue = []
            num_links += 1
            for n_start, n_curr in queue:
                # Find all nodes that are connected to the "current" node.
                connected_nodes = network[n_curr]
                for n_next in connected_nodes:
                    node_pair = (n_start, n_next)
                    if node_pair not in seen:
                        seen[node_pair] = num_links
                        next_queue.append([n_start, n_next])
            queue = next_queue

        # From all of the identified links, find out those with odd number of links
        odd_pairs = dict()
        for (n1,n2), v in seen.items():
            if v %2 == 1:
                # Make sure n1 < n2, to avoid duplications.
                if (n2 < n1):
                    n1, n2 = n2, n1 
                odd_pairs[(n1, n2)] = v

        # Return the number of odd links
        return len(odd_pairs)

    def solution_v2(self, A: List[int], B: List[int]) -> int:
        """Treat it as tree. Compile the results from children nodes.
        We may treat any node as the root node.
        """       
        
        def dfs(n:int, network: dict, seen: set, odd_links) -> List[int]:
            """Get "direct" links from children.
            Find out all posible inter-children links.
            Then return direct links (including self) back to the parent."""
            seen.add(n)
            children = network[n]
            direct_links = []
            return_links = []
            
            # Handle direct links
            for c in children:
                if c not in seen:
                    links = dfs(c, network, seen, odd_links)
                    for x in links:
                        if x % 2 == 1:
                            odd_links.append(x)
                        return_links.append(x + 1)
                    direct_links.append(links)
                    
            # Handle cross links (process two pairs at a time)
            for p in combinations(direct_links, 2):
                list1 = p[0]
                list2 = p[1]
                for x in list1:
                    for y in list2:
                        z = x + y
                        if z % 2 == 1:
                            odd_links.append(z)
                               
            # Add self to the reslt links
            return_links.append(1)         
            return return_links
            
        # Build the network: {node: [node]}
        network = defaultdict(set)
        for a, b in zip(A, B):
            network[a].add(b)
            network[b].add(a)
            
        odd_links = []
        seen = set()
        dfs(0, network, seen, odd_links)
        return len(odd_links)
            
            
def main():
    """Main function"""
    test_data = [
        [[0,3,4,2,6,3], [3,1,3,3,3,5], 6],
        [[0, 4, 2, 2, 4], [1, 3, 1, 3, 5], 9],
        [[0, 4, 4, 2, 7, 6, 3], [3, 5, 1, 3, 4, 3, 4], 16],
    ]
    ob1 = Solution()
    for A, B, ans in test_data:
        print(f"# Input: A={A}, B={B} (ans={ans})")
        print(f"  Output v1 = {ob1.solution_v1(A, B)}")
        print(f"  Output v2 = {ob1.solution_v2(A, B)}")

        
main()


# Input: A=[0, 3, 4, 2, 6, 3], B=[3, 1, 3, 3, 3, 5] (ans=6)
  Output v1 = 6
  Output v2 = 6
# Input: A=[0, 4, 2, 2, 4], B=[1, 3, 1, 3, 5] (ans=9)
  Output v1 = 9
  Output v2 = 9
# Input: A=[0, 4, 4, 2, 7, 6, 3], B=[3, 5, 1, 3, 4, 3, 4] (ans=16)
  Output v1 = 16
  Output v2 = 16
